In [8]:
# Run some setup code for this notebook.

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
# Load the raw CIFAR-10 data.
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
from cs231n.classifiers.linear_svm import svm_loss_naive
from cs231n.classifiers import LinearSVM
import time

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of about 0.4 on the validation set.

# need to test more carafully?????
lr_step = 1e-7
reg_step = 1e4
learning_rates = [1e-7, 5e-5]
regularization_strengths = [5e4, 1e5]
learning_rates = np.arange(1,501,50)*lr_step
#print learning_rates
regularization_strengths = np.arange(5,51,5)*reg_step
#print regularization_strengths
# results is dictionary mapping tuples of the form
# (learning_rate, regularization_strength) to tuples of the form
# (training_accuracy, validation_accuracy). The accuracy is simply the fraction
# of data points that are correctly classified.
results = {}
best_val = -1   # The highest validation accuracy that we have seen so far.
best_svm = None # The LinearSVM object that achieved the highest validation rate.


# other hyperparameters????
num_iters = 100 # default value actually
# how about batch_size??? # default value 200

# reload data
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# Preprocessing: reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

# As a sanity check, print out the shapes of the data
print 'Training data shape: ', X_train.shape
print 'Test data shape: ', X_test.shape

# @@@@ without subtraction, the accuracy is terriblely bad.@@@@
# subtract 
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image

X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
print 'After hstack, Training data shape: ', X_train.shape
print 'After hstack, Test data shape: ', X_test.shape

# validation set size is 1000, so get num_folds = 50000 / 1000
num_folds = 5

# init some values
val_accuracy = 0.0
tr_accuracy = 0.0
val_accuracy_sum = 0.0
tr_accuracy_sum = 0.0

X_train_folds = np.array_split(X_train,num_folds)
y_train_folds = np.array_split(y_train,num_folds)

# while(reg<regularization_strengths[1]):
#     while(lr<learning_rates[1]):

tic = time.time()
fo = open("validation_output.txt", "ab+")
for lr in learning_rates:
    for reg in regularization_strengths:
        svm = LinearSVM()
        val_accuracy_sum = 0.0
        tr_accuracy_sum = 0.0
        for i in range(num_folds):
            X_tr = X_train_folds[0:i] + X_train_folds[(i+1):num_folds]
            X_tr = np.reshape(X_tr,(X_train.shape[0]*(num_folds-1)/num_folds,-1))
            y_tr = y_train_folds[0:i] + y_train_folds[(i+1):num_folds]
            y_tr = np.reshape(y_tr,(X_train.shape[0]*(num_folds-1)/num_folds))
            X_val = np.reshape(X_train_folds[i],(X_train.shape[0]*1/num_folds,-1))
            
            loss_hist_tr = svm.train(X_tr,y_tr,learning_rate=lr,reg=reg,num_iters=num_iters,verbose=False)
            y_train_pred = svm.predict(X_tr.T)
            y_val_pred = svm.predict(X_val.T)
            val_accuracy = np.mean(y_train_folds[i] == y_val_pred)
            tr_accuracy = np.mean(y_tr == y_train_pred)
            str1 = '@@@@@validation@@@@@ lr %e reg %e train accuracy: %f val accuracy: %f' % (lr, reg, tr_accuracy, val_accuracy)
            print str1
            val_accuracy_sum += val_accuracy
            tr_accuracy_sum += tr_accuracy
            
        str2 = "End one validation.....\n"
        print str2
        val_accuracy = val_accuracy_sum/float(num_folds)
        tr_accuracy = tr_accuracy_sum/float(num_folds)
        if best_val < val_accuracy:
            best_val = val_accuracy
            best_svm = svm
        results[(lr,reg)] = (tr_accuracy,val_accuracy)
        str3 = "one combination...." + str(results[(lr,reg)]) + "\n"
        print str3
        str_write = str1 + "\n" + str2 + "\n" + str3 + "\n"
        fo.writelines(str_write)
fo.close()
toc = time.time()
print 'That took %fs' % (toc - tic)
################################################################################
# TODO:                                                                        #
# Write code that chooses the best hyperparameters by tuning on the validation #
# set. For each combination of hyperparameters, train a linear SVM on the      #
# training set, compute its accuracy on the training and validation sets, and  #
# store these numbers in the results dictionary. In addition, store the best   #
# validation accuracy in best_val and the LinearSVM object that achieves this  #
# accuracy in best_svm.                                                        #
#                                                                              #
# Hint: You should use a small value for num_iters as you develop your         #
# validation code so that the SVMs don't take much time to train; once you are #
# confident that your validation code works, you should rerun the validation   #
# code with a larger value for num_iters.                                      #
################################################################################
pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print 'lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy)
    
print 'best validation accuracy achieved during cross-validation: %f' % best_val

Training data shape:  (50000, 3072)
Test data shape:  (10000, 3072)
After hstack, Training data shape:  (50000, 3073)
After hstack, Test data shape:  (10000, 3073)
@@@@@validation@@@@@ lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.229000 val accuracy: 0.228100
@@@@@validation@@@@@ lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.288025 val accuracy: 0.276700
@@@@@validation@@@@@ lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.322125 val accuracy: 0.320600
@@@@@validation@@@@@ lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.351475 val accuracy: 0.357600
@@@@@validation@@@@@ lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.362625 val accuracy: 0.359600
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.31064999999999998, 0.30852000000000002)

@@@@@validation@@@@@ lr 1.000000e-07 reg 1.000000e+05 train accuracy: 0.246925 val accuracy: 0.250000
@@@@@validation@@@@@ lr 1.000000e-07 reg 1.000000e+05 train accuracy: 0.331250 val accuracy: 0.315400
@@@@@va

cs231n/classifiers/linear_svm.py:95: RuntimeWarning: overflow encountered in double_scalars
  loss += 0.5 * reg * np.sum(W*W)


@@@@@validation@@@@@ lr 1.010000e-05 reg 3.000000e+05 train accuracy: 0.082875 val accuracy: 0.079600
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.082220000000000001, 0.082220000000000001)

@@@@@validation@@@@@ lr 1.010000e-05 reg 3.500000e+05 train accuracy: 0.056375 val accuracy: 0.056300
@@@@@validation@@@@@ lr 1.010000e-05 reg 3.500000e+05 train accuracy: 0.056200 val accuracy: 0.057000
@@@@@validation@@@@@ lr 1.010000e-05 reg 3.500000e+05 train accuracy: 0.057075 val accuracy: 0.053500


cs231n/classifiers/linear_svm.py:95: RuntimeWarning: overflow encountered in multiply
  loss += 0.5 * reg * np.sum(W*W)


@@@@@validation@@@@@ lr 1.010000e-05 reg 3.500000e+05 train accuracy: 0.055600 val accuracy: 0.059400
@@@@@validation@@@@@ lr 1.010000e-05 reg 3.500000e+05 train accuracy: 0.056550 val accuracy: 0.055600
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.05636, 0.05636)

@@@@@validation@@@@@ lr 1.010000e-05 reg 4.000000e+05 train accuracy: 0.074600 val accuracy: 0.073600
@@@@@validation@@@@@ lr 1.010000e-05 reg 4.000000e+05 train accuracy: 0.074175 val accuracy: 0.075300
@@@@@validation@@@@@ lr 1.010000e-05 reg 4.000000e+05 train accuracy: 0.075025 val accuracy: 0.071900
@@@@@validation@@@@@ lr 1.010000e-05 reg 4.000000e+05 train accuracy: 0.073675 val accuracy: 0.077300
@@@@@validation@@@@@ lr 1.010000e-05 reg 4.000000e+05 train accuracy: 0.074525 val accuracy: 0.073900
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.074399999999999994, 0.074399999999999994)

@@@@@validation@@@@@ lr 1.010000e-05 reg 4.500000e+05 train accuracy: 0.069300

cs231n/classifiers/linear_svm.py:102: RuntimeWarning: overflow encountered in multiply
  dW += reg * W
cs231n/classifiers/linear_svm.py:85: RuntimeWarning: invalid value encountered in subtract
  margins = scores - scores_correct + 1.0
cs231n/classifiers/linear_svm.py:92: RuntimeWarning: invalid value encountered in less_equal
  margins[margins <= 0] = 0.0
cs231n/classifiers/linear_svm.py:98: RuntimeWarning: invalid value encountered in greater
  margins[margins>0] = 1.0


@@@@@validation@@@@@ lr 1.510000e-05 reg 3.500000e+05 train accuracy: 0.099650 val accuracy: 0.101400
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.069150000000000003, 0.069800000000000001)

@@@@@validation@@@@@ lr 1.510000e-05 reg 4.000000e+05 train accuracy: 0.051500 val accuracy: 0.050000
@@@@@validation@@@@@ lr 1.510000e-05 reg 4.000000e+05 train accuracy: 0.051475 val accuracy: 0.050100
@@@@@validation@@@@@ lr 1.510000e-05 reg 4.000000e+05 train accuracy: 0.051425 val accuracy: 0.050300
@@@@@validation@@@@@ lr 1.510000e-05 reg 4.000000e+05 train accuracy: 0.050325 val accuracy: 0.054700
@@@@@validation@@@@@ lr 1.510000e-05 reg 4.000000e+05 train accuracy: 0.099650 val accuracy: 0.101400
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.060874999999999999, 0.0613)

@@@@@validation@@@@@ lr 1.510000e-05 reg 4.500000e+05 train accuracy: 0.076675 val accuracy: 0.071500
@@@@@validation@@@@@ lr 1.510000e-05 reg 4.500000e+05 train accura

cs231n/classifiers/linear_classifier.py:67: RuntimeWarning: invalid value encountered in add
  self.W += -learning_rate * grad # 负号代表梯度“下降”


@@@@@validation@@@@@ lr 2.510000e-05 reg 4.000000e+05 train accuracy: 0.099925 val accuracy: 0.100300
@@@@@validation@@@@@ lr 2.510000e-05 reg 4.000000e+05 train accuracy: 0.099650 val accuracy: 0.101400
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.087705000000000005, 0.087139999999999995)

@@@@@validation@@@@@ lr 2.510000e-05 reg 4.500000e+05 train accuracy: 0.078675 val accuracy: 0.075200
@@@@@validation@@@@@ lr 2.510000e-05 reg 4.500000e+05 train accuracy: 0.077750 val accuracy: 0.078900
@@@@@validation@@@@@ lr 2.510000e-05 reg 4.500000e+05 train accuracy: 0.077550 val accuracy: 0.079700
@@@@@validation@@@@@ lr 2.510000e-05 reg 4.500000e+05 train accuracy: 0.099925 val accuracy: 0.100300
@@@@@validation@@@@@ lr 2.510000e-05 reg 4.500000e+05 train accuracy: 0.099650 val accuracy: 0.101400
End one validation.....

<type 'type'>
<type 'str'>
one combination....(0.086709999999999995, 0.087099999999999997)

@@@@@validation@@@@@ lr 2.510000e-05 reg 5.000000e+0